# Econometrics Assignment 3
-----

**Task 1: Do not use a built-in function to recalcuate the standard errors. **

a. Estimate the model with OLS, and report the coefficients, standard errors, t-stats and variance and covariance matrix of the coefficients. <br>
b. Calculate Heteroscedasticity-consistent standard errors, and reports t-stats with the Heteroscedasticity-consistent standard errors. 

**Task 2: Repeat task 1 with a built-in function that controls for Heteroscedasticity.. **

In [2]:
library(knitr); library(lmtest); library(sandwich)
options(scipen=100, digits=4)
opts_chunk$set(fig.path='figure/', fig.align='center', fig.show='hold')
options(formatR.arrow=TRUE,width=90)

# Download Data

In [3]:
mydat<-read.table(
  "http://www.kellogg.northwestern.edu/faculty/petersen/htm/papers/se/test_data.txt",
   col.names=c("firm", "year","x", "y"))
attach(mydat)

In [4]:
head(mydat);str(mydat)

firm,year,x,y
1,1,-1.113973,2.2515
1,2,-0.080854,1.2423
1,3,-0.237607,-1.4264
1,4,-0.152486,-1.1094
1,5,-0.001426,0.9147
1,6,-1.212737,-1.4247


'data.frame':	5000 obs. of  4 variables:
 $ firm: int  1 1 1 1 1 1 1 1 1 1 ...
 $ year: int  1 2 3 4 5 6 7 8 9 10 ...
 $ x   : num  -1.11397 -0.08085 -0.23761 -0.15249 -0.00143 ...
 $ y   : num  2.252 1.242 -1.426 -1.109 0.915 ...


# Task 1.
----
a. Estimate the model with OLS, and report the coefficients, standard errors, t-stats and variance and covariance matrix of the coefficients. <br>
b. Calculate Heteroscedasticity-consistent standard errors, and reports t-stats with the Heteroscedasticity-consistent standard errors.

### Task 1.(a) is easy. I use lm~ function to estimate OLS.

In [5]:
ols = lm(y~x)
summary(ols)


Call:
lm(formula = y ~ x)

Residuals:
   Min     1Q Median     3Q    Max 
-6.761 -1.368 -0.017  1.339  8.678 

Coefficients:
            Estimate Std. Error t value            Pr(>|t|)    
(Intercept)   0.0297     0.0284    1.05                 0.3    
x             1.0348     0.0286   36.20 <0.0000000000000002 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 2.01 on 4998 degrees of freedom
Multiple R-squared:  0.208,	Adjusted R-squared:  0.208 
F-statistic: 1.31e+03 on 1 and 4998 DF,  p-value: <0.0000000000000002


### I store OLS results for further comparison.

In [6]:
result = t(as.data.frame(summary(ols)$coefficients[2,1:3]))
row.names(result) = c("ols")

### For task1.(b), I calculate White standard error, and report t-stats.
----
**First**, I extract residuals from above OLS formula.<br>
**Second**, I put all the squares of the residuals into the diagonal parts of the n by n large matrix, where n equals to the length of variable.<br>
**Third**, I calculate variance using White's method, that is, 
$$var(\hat{\beta}) = (X'X)^{-1} X'\hat{\Omega}X (X'X)^{-1} $$
where the diagonal elements of $\hat{\Omega}$ eqals to the square value of OLS residuals, i.e., $\hat{\Omega_{ii}} = \hat{\epsilon_i}^2$ and $\hat{\Omega_{ij}} = 0$ for $i\neq j$.

In [7]:
e = residuals(ols)
omega = matrix(0, length(x), length(x))
diag(omega) = e^2

In [8]:
omega[1:10, 1:10]; str(omega)

11.39,0.00,0.000,0.0000,0.0000,0.00000,0.0000,0.000,0.000,0.00
0.00,1.68,0.000,0.0000,0.0000,0.00000,0.0000,0.000,0.000,0.00
0.00,0.00,1.465,0.0000,0.0000,0.00000,0.0000,0.000,0.000,0.00
0.00,0.00,0.000,0.9629,0.0000,0.00000,0.0000,0.000,0.000,0.00
0.00,0.00,0.000,0.0000,0.7859,0.00000,0.0000,0.000,0.000,0.00
0.00,0.00,0.000,0.0000,0.0000,0.03975,0.0000,0.000,0.000,0.00
0.00,0.00,0.000,0.0000,0.0000,0.00000,0.7413,0.000,0.000,0.00
0.00,0.00,0.000,0.0000,0.0000,0.00000,0.0000,5.681,0.000,0.00
0.00,0.00,0.000,0.0000,0.0000,0.00000,0.0000,0.000,1.632,0.00
0.00,0.00,0.000,0.0000,0.0000,0.00000,0.0000,0.000,0.000,7.85


 num [1:5000, 1:5000] 11.4 0 0 0 0 ...


In [9]:
(white.var = sum(omega*x^2)/(sum(x^2))^2)
(white = sqrt(white.var))

[1] 0.0008059

[1] 0.02839

### Finally, I report White standard error and t-stats with the comparison of OLS.

In [10]:
white.coef = summary(ols)$coefficients[2,c("Estimate")]
white.se = white
white.t = white.coef/white.se

result = rbind(result, c(white.coef, white.se, white.t))
rownames(result)[2] = c("white")

In [11]:
result

,Estimate,Std. Error,t value
ols,1.035,0.02858,36.20
white,1.035,0.02839,36.45


# Task 2.
----
Let's use buit-in function!

In [12]:
(white.builtin = coeftest(ols, vcov = function(x) vcovHC(x, method="white1", type="HC1")))


t test of coefficients:

            Estimate Std. Error t value            Pr(>|t|)    
(Intercept)   0.0297     0.0284    1.05                 0.3    
x             1.0348     0.0284   36.44 <0.0000000000000002 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [13]:
result = rbind(result, white.builtin[2,c("Estimate", "Std. Error", "t value")])
row.names(result)[3] = c("white.builtin")

# This is the final results.

In [14]:
result

,Estimate,Std. Error,t value
ols,1.035,0.02858,36.20
white,1.035,0.02839,36.45
white.builtin,1.035,0.02840,36.44


Contact Info: Jinkyu Kim, Business School, Hanyang Univ. email:jkyu126@gmail.com